In [ ]:
import pyspark

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('dodod').getOrCreate()

In [ ]:
df = spark.read.csv('hdfs://localhost:9000/Python/customer_churn.csv',inferSchema=True, header=True)

In [ ]:
df.show()

In [ ]:
df.printSchema()

In [ ]:
df.describe().show()

In [ ]:
df.columns

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',], outputCol= 'features')

In [ ]:
output = assembler.transform(df)

In [ ]:
final_df = output.select('features', 'churn')

In [ ]:
train_df, test_df = final_df.randomSplit([0.7,0.3])

In [14]:
train_df.describe().show()

+-------+-------------------+
|summary|              churn|
+-------+-------------------+
|  count|                613|
|   mean|0.15823817292006526|
| stddev|0.36526223283625364|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [15]:
test_df.describe().show()

+-------+-------------------+
|summary|              churn|
+-------+-------------------+
|  count|                287|
|   mean|0.18466898954703834|
| stddev| 0.3887065855235733|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [16]:
from pyspark.ml.classification import LogisticRegression

In [17]:
logr = LogisticRegression(labelCol='churn')

In [18]:
fitted_churn = logr.fit(train_df)

In [19]:
training_sum = fitted_churn.summary

In [20]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                613|                613|
|   mean|0.15823817292006526|0.11908646003262642|
| stddev|0.36526223283625364| 0.3241547288935984|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [21]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [22]:
pred_and_labels = fitted_churn.evaluate(test_df)

In [23]:
his = pred_and_labels.predictions

In [24]:
his.select('probability').show()

+--------------------+
|         probability|
+--------------------+
|[0.99317681866092...|
|[0.99875521967128...|
|[0.98704415309114...|
|[0.95088704025185...|
|[0.99214131627150...|
|[0.98730243217396...|
|[0.98645847250398...|
|[0.94162514442180...|
|[0.95418673314915...|
|[0.99896203656928...|
|[0.99100262285782...|
|[0.77108459286312...|
|[0.99906775558865...|
|[0.97160433023932...|
|[0.89618238656362...|
|[0.98173334254828...|
|[0.77433732661479...|
|[0.99913180166652...|
|[0.97382524032792...|
|[0.99714473663461...|
+--------------------+
only showing top 20 rows



In [25]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')

In [26]:
acc = churn_eval.evaluate(pred_and_labels.predictions)

In [27]:
acc

0.742783422028705

In [28]:
final_lr_model = logr.fit(train_df)

In [29]:
new_customers =spark.read.csv('hdfs://localhost:9000/Python/new_customers.csv', inferSchema=True, header=True)

In [30]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [31]:
new_customers.describe().show()

+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|summary|        Names|               Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|         Company|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|  count|            6|                 6|                6|                 6|                6|                 6|                   6|               6|
|   mean|         null|35.166666666666664|7607.156666666667|0.8333333333333334|6.808333333333334|12.333333333333334|                null|            null|
| stddev|         null| 15.71517313511584|4346.008232825459| 0.408248290463863|3.708737880555414|3.3862466931200785|                null|            null|
|    min|Andrew Mccall|              22.0|            100.0|          

In [32]:
test_new_customers = assembler.transform(new_customers)

In [33]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [34]:
final_results = final_lr_model.transform(test_new_customers)

In [35]:
final_results.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|            features|       rawPrediction|         probability|prediction|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|[37.0,9935.53,1.0...|[2.33210537083615...|[0.91150131719316...|       0.0|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|[23.0,7526.94,1.0...|[-5.8720828674019...|[0.00280908658590...|       

In [36]:
final_results.select('Names', 'Company', 'prediction').show()

+--------------+----------+
|         Names|prediction|
+--------------+----------+
| Andrew Mccall|       0.0|
|Michele Wright|       1.0|
|  Jeremy Chang|       1.0|
|Megan Ferguson|       1.0|
|  Taylor Young|       0.0|
| Jessica Drake|       1.0|
+--------------+----------+



In [37]:
test_new_customers.describe().show()

+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|summary|        Names|               Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|         Company|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|  count|            6|                 6|                6|                 6|                6|                 6|                   6|               6|
|   mean|         null|35.166666666666664|7607.156666666667|0.8333333333333334|6.808333333333334|12.333333333333334|                null|            null|
| stddev|         null| 15.71517313511584|4346.008232825459| 0.408248290463863|3.708737880555414|3.3862466931200785|                null|            null|
|    min|Andrew Mccall|              22.0|            100.0|          